In [12]:
from gensim.models import Word2Vec
import gensim
import jieba
import pymongo

In [10]:
model = gensim.models.Word2Vec.load('resume_model.model')

In [13]:
def jieba_cut(data, stop_words): # 定義使用結巴程式
    '''
    input:
        data : string
        stop_words: list of stopwords
    output:
        lst_seg : list of words after jieba
    '''

    # 使用結巴斷詞，產生 list of words
    seg_result = jieba.cut(data, cut_all=False)

    # 篩選斷詞，去掉單一中文字
    lst_seg = []

    for i in list(seg_result):
        i = i.strip()
        if len(i) < 1:  # 排除空值
            continue
        elif isEnglish(i) == False and len(i) == 1:  # 排除單一中文字
            continue
        elif i.isdigit() == True:  # 排除數字
            continue
        elif i in stop_words:  # 排除stopwords
            continue

        else:
            lst_seg.append(i)

    return lst_seg

In [14]:
def create_joblist_for_similar(job_list,model):
    user_list = []
    for word in job_list:
        if word in model.wv.vocab.keys():
            user_list.append(word)
        else:
            pass
    return user_list

In [15]:
def mongo_connect_build(db_name, col_name):
    
    global mycol
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") 

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    mycol = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立

def data_find():
    # 尋找多筆資料
    return mycol.find({},{'_id':0})

In [16]:
def open_json_file(CACHE_FNAME):
    try:
        cache_file = open(CACHE_FNAME, 'r', encoding='utf-8-sig')
        cache_contents = cache_file.read()
        CACHE_DICTION = json.loads(cache_contents, encoding='utf-8-sig')
        cache_file.close()
        return CACHE_DICTION

    except:
        CACHE_DICTION = {}
        return CACHE_DICTION

In [19]:
'''
丟入要匹配的user_list,
用model.n_similarity相似度匹配,
用user_list配對每一筆cv_list,求出相似度建立字典
'''
def job_map_cv(cv_dict,model,user_list):
    rate_dict = {}
    for k,v in zip(cv_dict.keys(),cv_prob.values()):    
        rate = model.n_similarity(v,user_list)
        rate_dict[k] = rate
        rate_sort = sorted(rate_dict.items(), key=lambda x:x[1],reverse=True)
        rate_list = list(rate_sort)
    return rate_list

In [20]:
def resume_select_url(rate_list):
    for i in range(0,10):
        print('https://www.cakeresume.com/' + str(rate_list[i][0]))
        print('============================================================')